In [1]:
import sys
sys.path.append(r'D:/projects/singletrader/')
# import singletrader
from singletrader.datautils.qlibapi.constructor.base import MultiFactor
import pandas as pd
import warnings
from workflow import MultiFactorTesting,bar_resample
from pprint import pprint
import numpy as np
warnings.filterwarnings('ignore')

{"level": "INFO", "time": "2023-03-05 21:16:20,426", "timeMillis": 1678022180072,"module": "numexpr.utils", "method": "python-service", "addition": "[utils.py 148 _init_num_threads]", "message": "Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8."}
{"level": "INFO", "time": "2023-03-05 21:16:20,427", "timeMillis": 1678022180072,"module": "numexpr.utils", "method": "python-service", "addition": "[utils.py 160 _init_num_threads]", "message": "NumExpr defaulting to 8 threads."}
{"level": "INFO", "time": "2023-03-05 21:16:20,735", "timeMillis": 1678022180072,"module": "singletrader-service", "method": "python-service", "addition": "[__init__.py 45 init]", "message": "开始初始化路径..."}
[22756:MainThread](2023-03-05 21:16:21,076) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
{"level": "INFO", "time": "2023-03-05 21:16:21,076", "timeMillis": 1678022180072,"module": "qlib.Initialization", "method": "python-service", "addition": "[

auth success 


In [2]:


fields = []
names = []


fields += ['$close','$open','$high','$low','$avg','$volume','$circulating_market_cap','$turnover_ratio','1/$pe_ratio']
bars = ['close','open','high','low','avg','volume','market_cap','turnover_ratio','ep']
names += bars


In [3]:
mf = MultiFactor(field=fields,name=names,start_date='2009-01-01',end_date='2022-12-31')
data = mf._data.swaplevel(0,1)
data.index = data.index.set_names(['date','asset'])

{"level": "INFO", "time": "2023-03-05 21:16:27,136", "timeMillis": 1678022180072,"module": "root", "method": "python-service", "addition": "[base.py 71 _data]", "message": "开始生成因子数据..."}


In [ ]:
## raw data (daily)
# 因为计算distance需要损失过去一年数据，所以原始数据从2009年开始

In [ ]:
# #原始数据示例
# data.head()

In [9]:
#收益月偏度计算
skew = data.groupby('asset').apply(lambda x:x['close'].droplevel('asset').pct_change().resample('M').apply(lambda x:x.skew()))
skew = skew.stack().swaplevel(0,1)
skew.name = 'skew'

#年度最高价距离计算 1 - close/Max(high,252)
distance = data.groupby('asset').apply(lambda x:(1-x['close'] / x['high'].rolling(252).max()).droplevel('asset').resample('M').last())
distance =  distance.stack().swaplevel(0,1)
distance.name = 'distance'


#行情数据降至月频率
bar_monthly = bar_resample(data[bars],frequency='M')

#3month动量
mom =  [bar_monthly.groupby('asset').apply(lambda x:x['close']/x['close'].shift(i)-1).droplevel(0) for i in range(1,13)]
mom = pd.concat(mom,axis=1)
mom.columns = ['mom%dM' % i for i in range(1,13)]


#3month turnover
turnover3M =  bar_monthly.groupby('asset').apply(lambda x:x['turnover_ratio'].rolling(3).sum()).droplevel(0)
turnover3M.name = 'turnover3M'


# #ep loc
# ep_loc =


#数据合并对齐
merged_data = pd.concat([bar_monthly, skew, distance, mom, turnover3M],axis=1)

In [11]:
merged_data.dropna()

,,close,open,high,low,avg,volume,market_cap,turnover_ratio,ep,skew,...,mom4M,mom5M,mom6M,mom7M,mom8M,mom9M,mom10M,mom11M,mom12M,turnover3M
date,asset,,,,,,,,,,,,,,,,,,,,,
2010-01-31,000001.XSHE,848.169983,958.390015,960.729980,805.169983,858.330017,24294088.0,634.532776,0.8231,0.013867,0.232038,...,0.084464,0.192976,-0.171118,-0.005499,0.216381,0.329670,0.361363,0.572461,0.864274,2.5215
2010-02-28,000001.XSHE,877.479980,848.559998,884.130005,820.809998,875.919983,10662390.0,656.463684,1.5397,0.013404,0.886190,...,-0.001332,0.121939,0.234201,-0.142475,0.028868,0.258415,0.375619,0.408407,0.626801,3.0973
2010-03-31,000001.XSHE,906.799988,877.479980,948.619995,866.539978,908.750000,15706730.0,678.394470,1.0449,0.069827,0.091379,...,-0.043692,0.032038,0.159428,0.275441,-0.113821,0.063246,0.300463,0.421584,0.455468,3.4077
2010-04-30,000001.XSHE,803.609985,909.530029,932.590027,754.750000,798.919983,20433608.0,601.197876,1.1092,0.085935,-1.859854,...,-0.156342,-0.152516,-0.085404,0.027490,0.130301,-0.214665,-0.057747,0.152476,0.259814,3.6938
2010-05-31,000001.XSHE,684.400024,785.630005,793.450012,665.239990,694.559998,18574160.0,512.012390,0.9249,0.100904,-0.454606,...,-0.193086,-0.281492,-0.278234,-0.221078,-0.124931,-0.037371,-0.331164,-0.197524,-0.018486,3.0790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,689009.XSHG,44.029999,49.849998,60.660000,42.900002,43.950001,81991272.0,224.061707,0.4342,0.013158,1.195883,...,0.184239,0.005481,-0.228221,-0.275941,-0.371628,-0.276299,-0.312031,-0.455748,-0.412933,1.9136
2022-09-30,689009.XSHG,38.310001,43.980000,45.200001,37.000000,38.299999,44163836.0,194.953506,0.3367,0.015091,0.500007,...,-0.159868,0.030393,-0.125143,-0.328484,-0.370005,-0.453261,-0.370316,-0.401406,-0.526452,1.1629
2022-10-31,689009.XSHG,32.000000,38.599998,43.990002,31.010000,32.099998,61281260.0,163.744095,1.0041,0.018230,-1.899364,...,-0.280899,-0.298246,-0.139322,-0.269240,-0.439089,-0.473771,-0.543314,-0.474030,-0.500000,1.7750


## Notes
universe: all A share

period: 2010.01~2022.12 (156 months)

$ forward\_return = \frac{close_{next\_month}}{open_{next\_month}}-1$

$ distance =  1 - close / MAX(high,252days)$

mom3M: currentclose / close 3 months ago

turnover3M: sum of last 3 months turnover ratio

market_cap: market capitalization in circulation

bechmark for equal weighted:  equal weight of all stock's forward return

bechmark for market Cap weighted:  market Cap weight of all stock's forward return

## merged data (monthly)
处理和合并后行情和因子值的月度数据，

In [ ]:
#合并数据示例
from pathlib import Path
from workflow import MultiFactorTesting
# __file__ = r'D:\projects\singletrader\samples\double_sort_framework\workflow.ipynb'
# file = Path(__file__)
# parent_path = file.parent
# data_file = parent_path.__str__() + '/data/' + r'price_and_factor_data.csv'
# merged_data = pd.read_csv(data_file)
# merged_data['date'] = pd.to_datetime(merged_data['date']) #date字段需要datetime格式；
# merged_data = merged_data.set_index(['date','asset']) #必须为date,asset双重索引；
merged_data = merged_data.dropna()
merged_data.dropna().head()

In [ ]:
from singletrader.factortesting.factor_testing import FactorEvaluation
features = ['ep','skew','distance','mom3W','turnover3W','market_cap']
fe = FactorEvaluation(merged_data[bars],merged_data[features],freq=12)

In [ ]:
normal_ic = fe.get_factor_ic(base='close-open',add_shift=0,method='normal')
rank_ic = fe.get_factor_ic(base='close-open',add_shift=0,method='rank')
res_ic = pd.DataFrame(columns=normal_ic.columns)

In [ ]:
res_ic.loc['ic.avg'] = normal_ic.mean()
res_ic.loc['ic.std'] = normal_ic.std()
res_ic.loc['icir'] = normal_ic.mean() / normal_ic.std()
res_ic.loc['rankic'] = rank_ic.mean()

In [ ]:
res_ic

In [ ]:
from singletrader.performance.common import performance_indicator
r = fe.get_group_returns(add_shift=0,groups=10,return_weight=True,base='close-open')
mkt_return=merged_data.dropna().groupby(level=0).apply(lambda x:(x['close']/x['open']-1).mean()).shift(-1)

In [ ]:
for factor in r[0].columns:
    print('different group return of '+factor,'\n')
    _per = performance_indicator((1+r[0][factor].unstack()).cumprod(),freq=12,language='en',mkt_return=mkt_return)
    _per.loc['turnover per trade'] = r[1][factor].groupby(level=0).apply(lambda x:x.diff().abs().sum(axis=1)/2).droplevel(0).unstack().T.mean()
    pprint(round(_per,2))
    print('\n')

In [ ]:
import plotly.express as px
for factor in r[0].columns:
    # print('different group return of '+factor,'\n')
    nv = (1+r[0][factor].unstack()).cumprod()
    p = px.line(nv,title='different group return of '+factor)
    p.show()


In [ ]:
from workflow import MultiFactorTesting

mft = MultiFactorTesting(bar_data=merged_data.dropna()[bars],factor_data=merged_data.dropna()[['market_cap','mom3W','turnover3W']],add_shift=0)
clean_factor_return = mft.get_clean_factor_return(quantiles=5)#clean_factor_return = mft.get_clean_factor_return(quantiles={'distance':3,'skew':3})
r = mft.compute_cell_return_eq(quantiles=5)
#因子分组和下期收益数据示例
# clean_factor_return.head()

In [ ]:
performance_indicator((1+r.unstack()).cumprod(),freq=12,language='en').T.sort_values('SR rf=0.02',ascending=False).head(20)

In [ ]:
clean_factor_return_copy = clean_factor_return.copy()
clean_factor_return_copy['next_return'] = 1
sort_count = clean_factor_return_copy.groupby(level=0).apply(lambda x:x.groupby(mft.all_factors).count()).unstack().unstack().droplevel(0,axis=1)

In [ ]:
sort_count.unstack().mean()

In [13]:
merged_data.to_csv(r'D:\projects\singletrader\samples\double_sort_framework\data\third_data.csv')